In [ ]:
import requests
from weasyprint import HTML
from pathlib import Path
import os

BASE_URL = "https://chilamboli.devmorphix.com"
ASSETS_DIR = "assets"
OUTPUT_DIR = "output"
OUTPUT_SUBDIR = "score sheet"
TEMPLATE_PATH = os.path.join(ASSETS_DIR, "scoring_sheet_template.html")
FONT_PATH = os.path.join(ASSETS_DIR, "Chilanka-Regular.ttf")

# Create output directories if they don't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR), exist_ok=True)

print(f"Config loaded: BASE_URL={BASE_URL}")
print(f"Assets directory: {ASSETS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Template: {TEMPLATE_PATH}")

In [ ]:
def fetch_event_registrations(event_id):
    """Fetch event and its registrations from API (for scoring sheet before event; no results/judgments required)"""
    url = f"{BASE_URL}/api/admin/events/{event_id}/registrations"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    data = response.json()
    
    if not data.get("success"):
        raise RuntimeError(data.get("message", "API error"))
    
    return data

def load_template():
    """Load HTML template"""
    if not os.path.exists(TEMPLATE_PATH):
        raise FileNotFoundError(f"Template file not found: {TEMPLATE_PATH}")
    with open(TEMPLATE_PATH, 'r', encoding='utf-8') as f:
        return f.read()

print("Helper functions defined: fetch_event_registrations, load_template")

In [ ]:
def escape_html(text):
    """Escape HTML special characters"""
    if text is None:
        return ""
    text = str(text)
    return (text.replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('"', '&quot;')
                .replace("'", '&#x27;'))

def create_scoring_sheet_pdf(event_data, output_filename, folder_name=None):
    """Create a PDF scoring sheet from HTML template (portrait, for one judge)."""
    event = event_data["event"]
    registrations = event_data["registrations"]
    template = load_template()
    
    # Build table rows: chest number, 5 blank columns, total (blank), portal tick (blank)
    table_rows = ""
    for reg in registrations:
        chest = escape_html(reg.get("chestNumber") or "-")
        row = '<tr class="results-data-row">'
        row += f'<td class="col-chest">{chest}</td>'
        for _ in range(5):
            row += '<td class="col-mark"></td>'
        row += '<td class="col-total"></td>'
        row += '<td class="col-portal"></td>'
        row += '</tr>'
        table_rows += row
    
    html_content = template.replace('{{event_name}}', escape_html(event.get("name", "")))
    html_content = html_content.replace('{{category}}', escape_html(event.get("ageCategory", "")))
    html_content = html_content.replace('{{event_type}}', escape_html(event.get("eventType", "")))
    html_content = html_content.replace('{{table_rows}}', table_rows)
    
    folder = folder_name if folder_name is not None else output_filename.replace('.pdf', '')
    report_folder_path = os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR, folder)
    os.makedirs(report_folder_path, exist_ok=True)
    pdf_path = os.path.join(report_folder_path, output_filename)
    html_path = os.path.join(report_folder_path, output_filename.replace('.pdf', '.html'))
    
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    HTML(string=html_content, base_url=Path(ASSETS_DIR).absolute()).write_pdf(pdf_path)
    print(f"✓ PDF generated: {pdf_path}")
    print(f"✓ HTML generated: {html_path}")

print("PDF generation function defined: create_scoring_sheet_pdf")

In [ ]:
# Generate scoring sheet for one event
EVENT_ID = input("Enter event ID: ").strip()
if not EVENT_ID:
    raise ValueError("Event ID is required. Please run the cell again and enter an event ID.")

print(f"Fetching registrations for event ID: {EVENT_ID}")
try:
    event_data = fetch_event_registrations(EVENT_ID)
    event_name = event_data["event"]["name"].replace(" ", "_").replace("/", "_")
    category = event_data["event"]["ageCategory"]
    folder_name = f"{category} - {event_name}"
    output_filename = f"{event_name}.pdf"
    create_scoring_sheet_pdf(event_data, output_filename, folder_name=folder_name)
    report_folder_path = os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR, folder_name)
    print(f"\n✓ Scoring sheet generated successfully!")
    print(f"  Event: {event_data['event']['name']}")
    print(f"  Category: {event_data['event']['ageCategory']}")
    print(f"  Rows: {len(event_data['registrations'])} registration(s)")
    print(f"  Output folder: {report_folder_path}")
    print(f"  PDF: {output_filename}")
except Exception as e:
    print(f"Error generating scoring sheet: {e}")
    import traceback
    traceback.print_exc()